In [22]:
# adapted from http://www.awebb.info/blog/observing_functions
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
#import utils
%matplotlib inline

# soft evidence for fact
f_1_of_10000 = pm.Categorical("f_1_of_10000", [0.1, 0.9])

# conditional probability for child
@pm.stochastic(observed=False)
#args == parents
def p_I_am_HIV_positive(value=1, f_1_of_10000=f_1_of_10000):
    # this is formed automatically from table or entered by user manually
    if (f_1_of_10000 == 1 and value == 1):
        p = 0.0001
    elif (f_1_of_10000 == 1 and value == 0):
        p = 0.9999
    elif (f_1_of_10000 == 0 and value == 1):
        p = 0.5
    elif (f_1_of_10000 == 0 and value == 0):
        p = 0.5
    else:
        return -np.inf
    return np.log(p)

# args = observed value and parents
# if observed value must be soft, then we add one more categerical parent - soft_evidence
soft_evidence = pm.Categorical("soft_evidence", [0.1, 0.9])
@pm.stochastic(observed=True)
def HIV_test_false_positive(value=1, soft_evidence=soft_evidence, p_I_am_HIV_positive=p_I_am_HIV_positive):
    # try value = soft_evidence
    if ( p_I_am_HIV_positive != 0 and p_I_am_HIV_positive != 1 ):
        return -np.inf
    if ( value != 0 and value != 1 ):
        return -np.inf
    
    if ( p_I_am_HIV_positive == 0):
        prob = 0.001 if soft_evidence == 1 else 0.999 # 0.001 - prob that test is true if a am not really HIV
    elif ( p_I_am_HIV_positive == 1):
        prob = 0.999 if soft_evidence == 1 else 0.001 # 0.999 - prob that test is true if a am really HIV
    else:
        prob = 0.5 # prob that test is true if a am not really HIV and my knowledge about test results is wrong
    return np.log(prob)
     
model = pm.Model([
    f_1_of_10000,
    soft_evidence,
    I_am_HIV_positive,
    p_I_am_HIV_positive,
    I_dont_know,
    HIV_test_false_positive
])
#graph = pm.graph.graph(model)
#Image(graph.create_png())
mcmc = pm.MCMC(model)
SAMPLE_NUM = 20000
mcmc.sample(SAMPLE_NUM, 2000)

alpha_samples = mcmc.trace("p_I_am_HIV_positive")[:]

print "%f" % (sum(alpha_samples)/float(SAMPLE_NUM))

 [-----------------100%-----------------] 20000 of 20000 complete in 27.6 sec0.450050
